In [57]:
import numpy as np
import pandas as pd

df = pd.read_csv(f'Subject_3/Shift_0/Motion_2_Rep_1.txt',skiprows=0,sep=' ',header=None)
rep_ = "1"
class_ = "2"
df.shape
df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,123,121,119,120,121,120,121,120,122,121,...,121,120,122,121,121,120,120,121,123,121
1,123,121,121,121,121,122,122,120,122,121,...,120,121,123,121,120,121,120,122,121,120
2,123,121,121,121,122,122,122,120,122,121,...,123,121,123,122,121,122,121,120,121,120
3,122,121,120,119,119,117,120,120,122,121,...,122,121,122,121,120,122,121,124,123,121
4,122,121,120,119,120,122,121,121,123,121,...,119,121,122,122,121,121,120,118,118,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,122,121,120,123,121,119,121,121,122,121,...,121,121,123,121,121,120,120,120,120,120
634,122,121,121,123,122,124,122,121,123,121,...,121,121,122,121,122,121,122,121,121,121
635,122,121,121,119,122,119,121,120,122,121,...,120,120,123,121,121,120,121,120,120,120
636,122,121,120,135,123,121,121,121,122,121,...,121,120,123,121,120,121,121,120,121,121


In [58]:
data_arr = np.stack([np.array(df.T[i::8]).T.flatten().astype('float32') for i in range (8)])
print(data_arr)
data_arr.shape

[[123. 122. 122. ... 123. 122. 122.]
 [121. 121. 121. ... 121. 121. 120.]
 [119. 120. 120. ... 120. 120. 120.]
 ...
 [120. 121. 121. ... 119. 117. 121.]
 [121. 121. 122. ... 120. 119. 120.]
 [120. 121. 121. ... 121. 120. 120.]]


(8, 10208)

In [59]:
def getFeatureMatrix(rawDataMatrix, windowLength, windowOverlap):
    rms = lambda sig: np.sqrt(np.mean(sig**2))
    nChannels,nSamples = rawDataMatrix.shape    
    I = int(np.floor(nSamples/(windowLength-windowOverlap)))
    featMatrix = np.zeros([nChannels, I])
    for channel in range(nChannels):
        for i in range (I):
            wdwStrtIdx=i*(windowLength-windowOverlap)
            sigWin = rawDataMatrix[channel][wdwStrtIdx:(wdwStrtIdx+windowLength-1)] 
            featMatrix[channel, i] = rms(sigWin)
    featMatrixData = np.array(featMatrix)
    return featMatrixData

In [60]:
Fs = 500
windowLength = int(np.floor(0.1*Fs))  #160ms
windowOverlap =  int(np.floor(50/100 * windowLength))
data_arr -= 121
data_arr /= 255.0
feaData = getFeatureMatrix(data_arr, windowLength, windowOverlap)

In [61]:
print(feaData)
feaData.shape

[[0.00560224 0.00551757 0.00537348 ... 0.00587568 0.005795   0.00550376]
 [0.00148221 0.00137226 0.00158455 ... 0.0012527  0.00112045 0.00136532]
 [0.0038407  0.00396138 0.00331433 ... 0.00340771 0.00367364 0.00354719]
 ...
 [0.00658114 0.00772216 0.0069522  ... 0.00590233 0.00363067 0.0047786 ]
 [0.0034986  0.0037581  0.00455128 ... 0.00340771 0.00306847 0.00348088]
 [0.0031192  0.00274453 0.00268674 ... 0.00296443 0.00296443 0.00297564]]


(8, 408)

In [62]:
rms_feature = feaData.sum(0)
print(rms_feature)
rms_feature.shape

[0.04235722 0.03915739 0.0408297  0.04225965 0.04575368 0.04932149
 0.04611981 0.04813135 0.04919435 0.04637601 0.05127576 0.04792696
 0.05289204 0.0597172  0.06874278 0.07977227 0.09033729 0.10702608
 0.10615753 0.0852076  0.08606961 0.09901585 0.0961855  0.10676473
 0.11086941 0.11480133 0.13156868 0.13847334 0.1532853  0.17336377
 0.15615453 0.13867647 0.19862984 0.1853526  0.14340221 0.16040325
 0.13917254 0.12054424 0.11255004 0.117499   0.14365637 0.13990038
 0.1464404  0.15889746 0.12757381 0.13045731 0.13383752 0.11886963
 0.11289256 0.11268164 0.12557773 0.11283925 0.12254038 0.1219753
 0.10459777 0.09891947 0.09346673 0.10335778 0.10646594 0.1118932
 0.13315313 0.12879463 0.11716747 0.13252794 0.15105814 0.14777765
 0.12664014 0.14631203 0.14785816 0.12752746 0.16560168 0.17865788
 0.18668081 0.21565385 0.26896906 0.26520457 0.25680021 0.2829915
 0.27363406 0.29640234 0.30064635 0.27734432 0.27390752 0.2663121
 0.28844309 0.28359767 0.27808274 0.28416238 0.28223008 0.28309902

(408,)

In [63]:
baseline = 2*rms_feature[-50:].mean()
print(baseline)


0.08867468455806375


In [64]:
start_ = np.argmax(rms_feature[::1]>baseline)
end_  = -np.argmax(rms_feature[::-1]>baseline)
print(start_)
print(end_)
feaData = feaData.T[start_:end_]
print(feaData)
print(feaData.shape)

16
-114
[[0.00674599 0.00345345 0.00367364 ... 0.02046161 0.02419364 0.00636292]
 [0.00708634 0.0034986  0.0034986  ... 0.02326111 0.0357228  0.00853308]
 [0.00692959 0.00321824 0.00403983 ... 0.0198702  0.0371695  0.00788305]
 ...
 [0.00648506 0.00291101 0.00321824 ... 0.0198623  0.03435229 0.00674599]
 [0.00633821 0.00296443 0.0038407  ... 0.01877378 0.03591993 0.00701959]
 [0.00643649 0.00316911 0.01338689 ... 0.01674124 0.01813604 0.00590233]]
(278, 8)


In [65]:
train_features = np.zeros([0,8])
train_labels = np.zeros([0])
test_features = np.zeros([0,8])
test_labels = np.zeros([0])
if rep_.startswith('2'):
    test_features = np.concatenate([test_features,feaData])
    test_labels = np.concatenate([test_labels,np.ones_like(feaData)[:,0]*int(class_)-1])
else:
    train_features = np.concatenate([train_features,feaData])
    train_labels = np.concatenate([train_labels,np.ones_like(feaData)[:,0]*int(class_)-1])

print(train_features)
print(train_features.shape)
print(train_labels)
print(train_labels.shape) 
print(test_features)
print(test_features.shape) 
print(test_labels)
print(test_labels.shape) 

[[0.00674599 0.00345345 0.00367364 ... 0.02046161 0.02419364 0.00636292]
 [0.00708634 0.0034986  0.0034986  ... 0.02326111 0.0357228  0.00853308]
 [0.00692959 0.00321824 0.00403983 ... 0.0198702  0.0371695  0.00788305]
 ...
 [0.00648506 0.00291101 0.00321824 ... 0.0198623  0.03435229 0.00674599]
 [0.00633821 0.00296443 0.0038407  ... 0.01877378 0.03591993 0.00701959]
 [0.00643649 0.00316911 0.01338689 ... 0.01674124 0.01813604 0.00590233]]
(278, 8)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [67]:
def roll_data(X_0, shift):
    """
    Roll (circular shift) the columns of a 2D numpy array.

    Args:
        X_0 (numpy.ndarray): Input 2D array.
        shift (int): Number of positions to shift the columns. Positive values shift to the left, negative values shift to the right.

    Returns:
        numpy.ndarray: The input array with its columns rolled (shifted).

    Example:
        >>> X_0 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
        >>> roll_data(X_0, 1)
        array([[3, 1, 2],
               [6, 4, 5],
               [9, 7, 8]])
    """
    return np.roll(X_0,shift,1)
def get_all_data(X, y, v_shift = None):
    """
    Generate augmented dataset by rolling the input data along columns.

    Args:
        X (list of numpy.ndarray): List of 2D numpy arrays representing input data.
        y (numpy.ndarray): 1D numpy array representing target labels.
        v_shift (int or list of int, optional): Vertical shift to apply to each input array. 
            If not provided, a range of shifts from -4 to 3 will be applied.

    Returns:
        tuple: A tuple containing three numpy arrays:
            - all_X: Augmented input data with rolled columns.
            - all_y: Corresponding target labels.
            - all_shift: Shift values applied to each sample.

    If v_shift is not provided, the function applies a range of shifts from -4 to 3 to the input data X.
    It rolls each input array along its columns and concatenates them into a single augmented dataset.
    The corresponding labels y are duplicated accordingly.
    The function returns three numpy arrays: all_X containing the augmented input data, all_y containing the corresponding labels,
    and all_shift containing the applied shift values for each sample.

    If v_shift is provided, the function applies the specified vertical shift to each input array in X.
    The labels and shift values are duplicated accordingly, and the function returns the concatenated arrays.

    Examples:
        >>> X = [np.array([[1, 2, 3], [4, 5, 6]]), np.array([[7, 8, 9], [10, 11, 12]])]
        >>> y = np.array([0, 1])
        >>> all_X, all_y, all_shift = get_all_data(X, y)
    """
    if not v_shift:
        all_X = np.zeros([X.shape[0],8, 8],dtype='float32') # 3D array: No. samples, 8: 8 features, 8: -4=>3 roll
        all_y = np.zeros([ y.shape[0],8],dtype='float32')
        all_shift = np.zeros([ y.shape[0],8],dtype='int')


        for i, shift in enumerate(range (-4,4)):
            X_i = roll_data(X, shift)
            all_X[:,i,:] = X_i
            all_y[:,i] = y
            all_shift[:,i] = shift
        return all_X.reshape(-1,8), all_y.reshape(-1,1), all_shift.reshape(-1,1)
    else:
        shapes = [X_.shape[0] for X_ in X]
        all_shift =[np.ones(shape, dtype='int')*v_shift for (v_shift, shape) in zip(v_shift,shapes)]
        return np.concatenate(X).reshape(-1,8), np.concatenate(y).reshape(-1,1), np.concatenate(all_shift).reshape(-1,1)
    

In [68]:
all_X_train, all_y_train, all_shift_train = get_all_data(train_features,train_labels)
print(all_X_train)
print(all_X_train.shape)
print(all_y_train)
print(all_y_train.shape)
print(all_shift_train)
print(all_shift_train.shape)

[[0.00590233 0.02046161 0.02419364 ... 0.00345345 0.00367364 0.01954371]
 [0.01954371 0.00590233 0.02046161 ... 0.00674599 0.00345345 0.00367364]
 [0.00367364 0.01954371 0.00590233 ... 0.00636292 0.00674599 0.00345345]
 ...
 [0.00590233 0.00643649 0.00316911 ... 0.00882242 0.01674124 0.01813604]
 [0.01813604 0.00590233 0.00643649 ... 0.02122946 0.00882242 0.01674124]
 [0.01674124 0.01813604 0.00590233 ... 0.01338689 0.02122946 0.00882242]]
(2224, 8)
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
(2224, 1)
[[-4]
 [-3]
 [-2]
 ...
 [ 1]
 [ 2]
 [ 3]]
(2224, 1)


In [69]:
def get_shift_data(all_X, all_shift, all_y):
    """
    Combine the original feature data with shifted feature data for each class.

    Args:
        all_X (numpy.ndarray): Original feature data matrix of shape (n_samples, n_features).
        all_shift (numpy.ndarray): Shifted feature data matrix of shape (n_samples, n_shift_features).
        all_y (numpy.ndarray): Array of class labels of shape (n_samples,).

    Returns:
        numpy.ndarray: Concatenated feature data matrix with original features for each class.
        numpy.ndarray: Concatenated feature data matrix with shifted features for each class.
        numpy.ndarray: Shift labels corresponding to the original feature data.
        numpy.ndarray: Shift labels corresponding to the shifted feature data.
        numpy.ndarray: Array of class labels for the concatenated data.

    This function combines the original feature data with shifted feature data for each class. It concatenates the
    original feature data (all_X) with the shifted feature data (all_shift) along the feature axis. It also shuffles
    the shifted feature data within each class to create a corresponding set of shifted features (all_X2).
    Additionally, it constructs arrays for the shift labels (all_shift_1 and all_shift_2) and the class labels (all_y_).
    The function returns all these arrays for further processing.

    Note:
        The function assumes that the original feature data (all_X) and the shifted feature data (all_shift) have the
        same number of samples and that each sample corresponds to a class label in the array all_y.
    """
    all_X_shift = np.concatenate([all_X, all_shift], axis=1)
    all_X1 = np.zeros_like(all_X)
    all_X2 = np.zeros_like(all_X)
    all_y_ = np.zeros_like(all_y)
    all_shift_1 = np.zeros_like(all_y)
    all_shift_2 = np.zeros_like(all_y)

    for class_label in range (9):
        class_idx = all_y.flatten() == class_label
        class_data = all_X_shift[class_idx]
        class_data_clone = class_data.copy()
        np.random.shuffle(class_data_clone)

        all_X1[class_idx] = class_data[:,:-1]
        all_X2[class_idx] = class_data_clone[:,:-1]
        all_shift_1[class_idx] = class_data[:,-1:]
        all_shift_2[class_idx] = class_data_clone[:,-1:]
        all_y_[class_idx] = class_label
    return all_X1, all_X2, all_shift_1, all_shift_2, all_y_

In [70]:
all_X1_train, all_X2_train, all_shift_1_train, all_shift_2_train, all_y_shift_train = get_shift_data(all_X_train, all_shift_train, all_y_train)

In [72]:
print(all_X1_train)
print(all_X1_train.shape)
print()
print(all_X2_train)
print(all_X2_train.shape)
print()
print(all_shift_1_train)
print(all_shift_1_train.shape)
print()
print(all_shift_2_train)
print(all_shift_2_train.shape)
print()
print(all_y_shift_train)
print(all_y_shift_train.shape)

[[0.00590233 0.02046161 0.02419364 ... 0.00345345 0.00367364 0.01954371]
 [0.01954371 0.00590233 0.02046161 ... 0.00674599 0.00345345 0.00367364]
 [0.00367364 0.01954371 0.00590233 ... 0.00636292 0.00674599 0.00345345]
 ...
 [0.00590233 0.00643649 0.00316911 ... 0.00882242 0.01674124 0.01813604]
 [0.01813604 0.00590233 0.00643649 ... 0.02122946 0.00882242 0.01674124]
 [0.01674124 0.01813604 0.00590233 ... 0.01338689 0.02122946 0.00882242]]
(2224, 8)

[[0.00747432 0.00605975 0.03058228 ... 0.06246185 0.01775124 0.010892  ]
 [0.01316229 0.00804073 0.00475366 ... 0.01134367 0.03131247 0.04835808]
 [0.02530948 0.01033002 0.02759353 ... 0.00772216 0.00478656 0.00510388]
 ...
 [0.00672269 0.02145007 0.03252196 ... 0.00358718 0.00475366 0.01570627]
 [0.00674599 0.00345345 0.00367364 ... 0.02046161 0.02419364 0.00636292]
 [0.01022313 0.03220192 0.06000416 ... 0.0051953  0.005795   0.02464986]]
(2224, 8)

[[-4.]
 [-3.]
 [-2.]
 ...
 [ 1.]
 [ 2.]
 [ 3.]]
(2224, 1)

[[-1.]
 [ 1.]
 [-3.]
 ...
 [-4.